## Load CORDEX Data from CDS  API

In [1]:
"""
This script makes calls to the CORDEX API

Data: CORDEX regional climate model data on single levels - Experiment: Historical
Temporal coverage: 1 Jan 1971 to 31 Dec 2000
Spatial coverage: Domain: Africa
Format: NetCDF in zip archives

Data: CORDEX regional climate model data on single levels - Experiment: RCP4.5
Temporal coverage: 1 Jan 2071 to 31 Dec 2100
Spatial coverage: Domain: Africa
Format: NetCDF in zip archives

"""

# CDS API
import cdsapi

# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# Other libraries (e.g. paths, filenames, zipfile extraction)
from glob import glob
from pathlib import Path
from os.path import basename
import zipfile
import yaml
import urllib3 
urllib3.disable_warnings() # Disable "InsecureRequestWarning" 
                           # for data download via API

import os
from dotenv import load_dotenv

load_dotenv()

URL = 'https://cds.climate.copernicus.eu/api/v2'
KEY = os.getenv('CDS_API_KEY')

def call_api(url=URL, key=KEY):
    client = cdsapi.Client(url=url, key=key)
    return client

def retrieve_cordex_hist_proj(client, data_dir):
    client.retrieve(
        'projections-cordex-domains-single-levels',
        {
            'format': 'zip',
            'domain': 'africa',
            'experiment': 'historical',
            'horizontal_resolution': '0_44_degree_x_0_44_degree',
            'temporal_resolution': 'daily_mean',
            'variable': '2m_air_temperature',
            'gcm_model': 'cccma_canesm2',
            'rcm_model': 'cccma_canrcm4',
            'ensemble_member': 'r1i1p1',
            'start_year': ['1971', '1976', '1981', '1986', '1991', '1996'],
            'end_year': ['1975', '1980', '1985', '1990', '1995', '2000'],
        },
        f'{data_dir}1971-2000_cordex_historical_africa.zip')
    
    client.retrieve(
        'projections-cordex-domains-single-levels',
        {
            'format': 'zip',
            'domain': 'africa',
            'experiment': 'rcp_4_5',
            'horizontal_resolution': '0_44_degree_x_0_44_degree',
            'temporal_resolution': 'daily_mean',
            'variable': '2m_air_temperature',
            'gcm_model': 'cccma_canesm2',
            'rcm_model': 'cccma_canrcm4',
            'ensemble_member': 'r1i1p1',
            'start_year': ['2071', '2076', '2081', '2086', '2091', '2096'],
            'end_year': ['2075', '2080', '2085', '2090', '2095', '2100'],
        },
        f'{data_dir}2071-2100_cordex_rcp_4_5_africa.zip')

def unzip_files(data_dir):
    cordex_zip_paths = glob(f'{data_dir}*.zip')
    for j in cordex_zip_paths:
        with zipfile.ZipFile(j, 'r') as zip_ref:
            zip_ref.extractall(f'{data_dir}')

def load_hist_proj(data_dir):
    hist_data = xr.open_mfdataset(f'{data_dir}*CanESM2_historical*.nc')
    proj_data = xr.open_mfdataset(f'{data_dir}*CanESM2_rcp45*.nc')
    return hist_data, proj_data


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
config_path = os.getenv('CONFIG_PATH', 'config.yaml')
# print(config_path)
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [3]:
data_dir = config['data_dir']

In [4]:
def main():
    data_dir = config['data_dir']
    client = call_api()
    retrieve_cordex_hist_proj(client, data_dir)
    unzip_files(data_dir)
    hist_data, proj_data = load_hist_proj(data_dir)

In [5]:
main()

2024-05-09 14:50:49,707 INFO Welcome to the CDS
2024-05-09 14:50:49,708 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cordex-domains-single-levels
2024-05-09 14:50:50,085 INFO Request is completed
2024-05-09 14:50:50,086 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data0/dataset-projections-cordex-domains-single-levels-3f1cee90-f50b-4536-b4c1-52d2fa802c1d.zip to ./data/1971-2000_cordex_historical_africa.zip (908.2M)
2024-05-09 15:04:52,639 INFO Download rate 1.1M/s    
2024-05-09 15:04:53,522 INFO Welcome to the CDS
2024-05-09 15:04:53,523 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cordex-domains-single-levels
2024-05-09 15:04:53,808 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data0/dataset-projections-cordex-domains-single-levels-8ab65b47-f78d-45c9-b739-efad62d81118.zip to ./data/2071-2100_cordex_rcp_4_5_afr

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [4]:
hist_data, proj_data = load_hist_proj(data_dir)
hist_data

<xarray.Dataset> Size: 2GB
Dimensions:       (time: 10950, rlon: 194, rlat: 201, bnds: 2)
Coordinates:
  * time          (time) object 88kB 1971-01-01 12:00:00 ... 2000-12-31 12:00:00
  * rlon          (rlon) float64 2kB -24.64 -24.2 -23.76 ... 59.4 59.84 60.28
  * rlat          (rlat) float64 2kB -45.76 -45.32 -44.88 ... 41.36 41.8 42.24
    lon           (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    lat           (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    height        float64 8B 2.0
Dimensions without coordinates: bnds
Data variables:
    rotated_pole  (time) |S1 11kB b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    tas           (time, rlat, rlon) float32 2GB dask.array<chunksize=(1, 201, 194), meta=np.ndarray>
    time_bnds     (time, bnds) object 175kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/26)
    title:                          CanRCM4 model output prepared for CORDEX ...
    institution:                    CCCma (Canadian Centre for Climate Modell...
    institute_id:                   CCCma
    contact:                        cccma_info@ec.gc.ca
    Conventions:                    CF-1.4
    experiment:                     Historical run driven by CCCma-CanESM2
    ...                             ...
    history:                        created: 2012-06-29 23:52:08 by rcm2nc
    data_licence:                   1) GRANT OF LICENCE - The Government of C...
    creation_date:                  2012-06-29-T23:47:03Z
    c3s_comment:                    This data has been published at ESGF with...
    tracking_id:                    hdl:21.14103/3d35a8d3-105d-451c-88fa-4d46...
    c3s_disclaimer:                 This data has been curated and prepared i...

In [6]:
tas_historical = hist_data['tas']
tas_historical

<xarray.DataArray 'tas' (time: 10950, rlat: 201, rlon: 194)> Size: 2GB
dask.array<concatenate, shape=(10950, 201, 194), dtype=float32, chunksize=(1, 201, 194), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 88kB 1971-01-01 12:00:00 ... 2000-12-31 12:00:00
  * rlon     (rlon) float64 2kB -24.64 -24.2 -23.76 -23.32 ... 59.4 59.84 60.28
  * rlat     (rlat) float64 2kB -45.76 -45.32 -44.88 -44.44 ... 41.36 41.8 42.24
    lon      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    lat      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    height   float64 8B 2.0
Attributes:
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K
    grid_mapping:   rotated_pole
    cell_methods:   time: mean

In [7]:
tas_climatology = tas_historical.groupby('time.month').mean()

In [9]:
tas_climatology_degC = tas_climatology - 273.15
tas_climatology_degC

<xarray.DataArray 'tas' (month: 12, rlat: 201, rlon: 194)> Size: 2MB
dask.array<sub, shape=(12, 201, 194), dtype=float32, chunksize=(1, 201, 194), chunktype=numpy.ndarray>
Coordinates:
  * rlon     (rlon) float64 2kB -24.64 -24.2 -23.76 -23.32 ... 59.4 59.84 60.28
  * rlat     (rlat) float64 2kB -45.76 -45.32 -44.88 -44.44 ... 41.36 41.8 42.24
    lon      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    lat      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    height   float64 8B 2.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [13]:
kiambere_tas_climatology_degC = tas_climatology_degC.sel(rlat=-0.7, rlon=37.783, method="nearest")
kiambere_tas_climatology_degC

<xarray.DataArray 'tas' (month: 12)> Size: 48B
dask.array<getitem, shape=(12,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    rlon     float64 8B 37.84
    rlat     float64 8B -0.88
    lon      float64 8B dask.array<chunksize=(), meta=np.ndarray>
    lat      float64 8B dask.array<chunksize=(), meta=np.ndarray>
    height   float64 8B 2.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [5]:
proj_data

<xarray.Dataset> Size: 2GB
Dimensions:       (time: 10950, rlon: 194, rlat: 201, bnds: 2)
Coordinates:
  * time          (time) object 88kB 2071-01-01 12:00:00 ... 2100-12-31 12:00:00
  * rlon          (rlon) float64 2kB -24.64 -24.2 -23.76 ... 59.4 59.84 60.28
  * rlat          (rlat) float64 2kB -45.76 -45.32 -44.88 ... 41.36 41.8 42.24
    lon           (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    lat           (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    height        float64 8B 2.0
Dimensions without coordinates: bnds
Data variables:
    rotated_pole  (time) |S1 11kB b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    tas           (time, rlat, rlon) float32 2GB dask.array<chunksize=(1, 201, 194), meta=np.ndarray>
    time_bnds     (time, bnds) object 175kB dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/26)
    title:                          CanRCM4 model output prepared for CORDEX ...
    institution:                    CCCma (Canadian Centre for Climate Modell...
    institute_id:                   CCCma
    contact:                        cccma_info@ec.gc.ca
    Conventions:                    CF-1.4
    experiment:                     RCP4.5 run driven by CCCma-CanESM2 
    ...                             ...
    history:                        created: 2012-06-29 23:55:49 by rcm2nc
    data_licence:                   1) GRANT OF LICENCE - The Government of C...
    creation_date:                  2012-06-29-T23:48:22Z
    c3s_comment:                    This data has been published at ESGF with...
    tracking_id:                    hdl:21.14103/7bd05374-ef0d-4b0a-80fc-5e08...
    c3s_disclaimer:                 This data has been curated and prepared i...

In [12]:
tas_projection = proj_data['tas']
tas_proj_climatology = tas_projection.groupby('time.month').mean()
tas_proj_climatology_degC = tas_proj_climatology - 273.15
tas_proj_climatology_degC

<xarray.DataArray 'tas' (month: 12, rlat: 201, rlon: 194)> Size: 2MB
dask.array<sub, shape=(12, 201, 194), dtype=float32, chunksize=(1, 201, 194), chunktype=numpy.ndarray>
Coordinates:
  * rlon     (rlon) float64 2kB -24.64 -24.2 -23.76 -23.32 ... 59.4 59.84 60.28
  * rlat     (rlat) float64 2kB -45.76 -45.32 -44.88 -44.44 ... 41.36 41.8 42.24
    lon      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    lat      (rlat, rlon) float64 312kB dask.array<chunksize=(201, 194), meta=np.ndarray>
    height   float64 8B 2.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [14]:
kiambere_tas_proj_climatology_degC = tas_proj_climatology_degC.sel(rlat=-0.7, rlon=37.783, method="nearest")
kiambere_tas_proj_climatology_degC

<xarray.DataArray 'tas' (month: 12)> Size: 48B
dask.array<getitem, shape=(12,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    rlon     float64 8B 37.84
    rlat     float64 8B -0.88
    lon      float64 8B dask.array<chunksize=(), meta=np.ndarray>
    lat      float64 8B dask.array<chunksize=(), meta=np.ndarray>
    height   float64 8B 2.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

In [15]:
kiambere_tas_difference_climatology = kiambere_tas_proj_climatology_degC - kiambere_tas_climatology_degC
kiambere_tas_difference_climatology

<xarray.DataArray 'tas' (month: 12)> Size: 48B
dask.array<sub, shape=(12,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    rlon     float64 8B 37.84
    rlat     float64 8B -0.88
    lon      float64 8B 37.84
    lat      float64 8B -0.88
    height   float64 8B 2.0
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

## Load Lat and Lon Data from Geocoding API

In [24]:
import requests
from requests.structures import CaseInsensitiveDict
import os
import json

from dotenv import load_dotenv

load_dotenv()

GEOCODE_API_KEY = os.getenv('GEOCODE_API')
location_query = "Kiambere"

url = f"https://api.geoapify.com/v1/geocode/search?text={location_query}&apiKey={GEOCODE_API_KEY}"
print(url)

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

response = resp.json()
lon = response['features'][0]['properties']['lon']
lat = response['features'][0]['properties']['lat']

print(lon, lat)
print(response)

https://api.geoapify.com/v1/geocode/search?text=Kiambere&apiKey=a450e2fe55d5448b82b97ef98174f6b4
37.7833 -0.7
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'name': 'Kiambere', 'country': 'Kenya', 'country_code': 'ke', 'state': 'Embu', 'county': 'Mbeere South', 'city': 'Kiambere', 'village': 'Kiambere', 'lon': 37.7833, 'lat': -0.7, 'result_type': 'city', 'formatted': 'Kiambere (Kiambere), Embu, Kenya', 'address_line1': 'Kiambere (Kiambere)', 'address_line2': 'Embu, Kenya', 'category': 'populated_place', 'timezone': {'name': 'Africa/Nairobi', 'offset_STD': '+03:00', 'offset_STD_seconds': 10800, 'offset_DST': '+03:00', 'offset_DST_seconds': 10800, 'abbreviation_STD': 'EAT', 'abbreviation_DST': 'EAT'}, 'plus_code': '6GFV7QXM+X8', 'rank': {'importance': 0.37500999999999995, 'popular

In [17]:
response = {
    'features'  :  [{'type': 'Feature', 'properties': {'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'name': 'Kiambere', 'country': 'Kenya', 'country_code': 'ke', 'state': 'Embu', 'county': 'Mbeere South', 'city': 'Kiambere', 'village': 'Kiambere', 'lon': 37.7833, 'lat': -0.7, 'result_type': 'city', 'formatted': 'Kiambere (Kiambere), Embu, Kenya', 'address_line1': 'Kiambere (Kiambere)', 'address_line2': 'Embu, Kenya', 'category': 'populated_place', 'timezone': {'name': 'Africa/Nairobi', 'offset_STD': '+03:00', 'offset_STD_seconds': 10800, 'offset_DST': '+03:00', 'offset_DST_seconds': 10800, 'abbreviation_STD': 'EAT', 'abbreviation_DST': 'EAT'}, 'plus_code': '6GFV7QXM+X8', 'rank': {'importance': 0.37500999999999995, 'popularity': 0.6095602004872759, 'confidence': 1, 'confidence_city_level': 1, 'match_type': 'full_match'}, 'place_id': '51787aa52c43e4424059666666666666e6bff00103f9018794ae0200000000c002089203084b69616d62657265'}, 'geometry': {'type': 'Point', 'coordinates': [37.7833, -0.7]}, 'bbox': [37.7633, -0.72, 37.8033, -0.68]}]
}

In [22]:
response['features'][0]['properties']['lon']
response['features'][0]['properties']['lat']


-0.7

## Conversation Agent

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [3]:
from requests.structures import CaseInsensitiveDict
from requests.exceptions import Timeout
import os
from dotenv import load_dotenv
import requests
from pydantic import BaseModel, Field
import datetime

# define the input schema
class LocationInput(BaseModel):
    location: str = Field(..., description="User entered location of interest")

@tool(args_schema=LocationInput)
def get_lat_lon(location: str):
    """
    Get the latitude and longitude of the location name given by user.
    """
    load_dotenv()

    GEOCODE_API_KEY = os.getenv('GEOCODE_API')
    location_query = location

    url = f"https://api.geoapify.com/v1/geocode/search?text={location_query}&apiKey={GEOCODE_API_KEY}"
    print(url)

    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)

    response = resp.json()
    lat = response['features'][0]['properties']['lat']
    lon = response['features'][0]['properties']['lon']

    return lat, lon


In [4]:
@tool
def get_soil_from_api(lat, lon):
    """
    Retrieves the soil type at a given latitude and longitude using the ISRIC SoilGrids API.

    Parameters:
    lat (float): The latitude of the location.
    lon (float): The longitude of the location.

    Returns:
    str: The name of the World Reference Base (WRB) soil class at the given location.
    """
    try:
        url = f"https://rest.isric.org/soilgrids/v2.0/classification/query?lon={lon}&lat={lat}&number_classes=5"
        response = requests.get(url, timeout=3)  # Set timeout to 2 seconds
        data = response.json()
        return data["wrb_class_name"]
    except Timeout:
        return "not found"

In [5]:
tools = [get_lat_lon, get_soil_from_api]

In [6]:
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
from ollama_functions import OllamaFunctions

In [15]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful data assistant"),
    ("user", "{input}"),
])

functions = [format_tool_to_openai_function(f) for f in tools]


model = OllamaFunctions(
    model="llama3", temperature=0
).bind_tools(functions)

chain = prompt | model | OpenAIFunctionsAgentOutputParser

In [11]:
# functions = [format_tool_to_openai_function(f) for f in tools]
# model = ChatOpenAI(temperature=0).bind(functions=functions)
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful data assistant"),
#     ("user", "{input}"),
# ])
# chain = prompt | model | OpenAIFunctionsAgentOutputParser

In [16]:
result = chain.invoke({"input": "what is the soil like in Nairobi?"})

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc182f8e1d0>: Failed to establish a new connection: [Errno 61] Connection refused'))